In [1]:
# Optional: mounting to Google Drive to read the data files.
from google.colab import drive
mount_path = '/content/gdrive'
drive.mount(mount_path)
notebook_root = 'gdrive/My Drive/mafat'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd '/content/gdrive/My Drive/mafat'
!pip install torch-stft

import sys
sys.path.append('/content/gdrive//My Drive/mafat')

from network_support import * 
from support_files import *

import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from math import pi as pi


# Public test set - loading and preprocessing
train_path = 'Phase 2 - Private/MAFAT RADAR Challenge - FULL Public Test Set V1'
test_df = load_data(train_path)
test_df = data_preprocess(test_df, mode = 0)
print("Done with train_df...")
describe_dict(test_df, "Test DF")
snrs = ['low_snr', 'high_snr']

/content/gdrive/.shortcut-targets-by-id/1CjXe8JAlTXTHOBVrITA52MQVVfPU_uah/mafat
Done with train_df...
######  Test DF  #########################################
Length: 284
Total 0 count: 138
Total 1 count: 146
Unique Devices: [ 1  4 14]
Shape of Images: (284, 126, 32)
###########################################################


#TESTING RESULTS OF MODELS
####To test the results of all trained models in the train_space, you need to know the folder to which the state was saved, the version number of the state, the state's save_name (which should be the name of the model along with a descrptive phrase of the variant), and the model handler. Ensure that all this information is correct in the variables specified. The output is the AUC ROC for each fold trained, along with the ensemble. 

In [11]:
for snr in snrs:
  
  ########################
  ########################      fix these variables to match the folder and states of interest
  batch_size = 32
  model_folder = "others"
  device = 'cuda'
  version_number = 1
  model_name = f"net6_{snr}"
  model = Net6().to(device)
  folds = 5
  ########################
  ########################

  print(f"TRAINING SNR TYPE: {snr}, MODEL: {model_name}\n")

  submission =  pd.DataFrame()
  submission['segment_id'] = test_df['segment_id']
  save_name = f'v{version_number}_{model_name}_final'

  for i in range(folds):
      submission[f'fold {i}'] = show_validation_accuracy_two(test_df['iq_sweep_burst'].reshape(list(test_df['iq_sweep_burst'].shape) + [1]), None, i, model_folder, save_name, model, batch_size, 0)
      out = pred_description(submission[f'fold {i}'], test_df['target_type'].astype(int), True)
      print(f"ROC-AUC: {out['ROC_AUC']}")
  
  submission['prediction'] = submission[[f'fold {j}' for j in range(folds)]].median(1).astype('float')
  out = pred_description(submission['prediction'], test_df['target_type'].astype(int), True)
  print(f"ROC-AUC: {out['ROC_AUC']}")
  print('\n')


TRAINING SNR TYPE: low_snr, MODEL: net6_low_snr

Number of Total Items: 284
ROC-AUC: 0.8426146515783205
Number of Total Items: 284
ROC-AUC: 0.8344748858447489
Number of Total Items: 284
ROC-AUC: 0.8392396267619616
Number of Total Items: 284
ROC-AUC: 0.8617728806829461
Number of Total Items: 284
ROC-AUC: 0.8721461187214612
Number of Total Items: 284
ROC-AUC: 0.8559162199722057


TRAINING SNR TYPE: high_snr, MODEL: net6_high_snr

Number of Total Items: 284
ROC-AUC: 0.746079015286877
Number of Total Items: 284
ROC-AUC: 0.783551717292039
Number of Total Items: 284
ROC-AUC: 0.7623089140361325
Number of Total Items: 284
ROC-AUC: 0.7949672424061942
Number of Total Items: 284
ROC-AUC: 0.7688604327973
Number of Total Items: 284
ROC-AUC: 0.7791840381179274




#TEST EFFNET:
####To test the EffNet trained for spectrogram classification, run the below cell. Similarlty to the procedure above, change the file and folder names of the EffNet state storage

In [4]:

!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

class EffNetModel(nn.Module):
    def __init__(
            self,
            efficient_net_type='efficientnet-b0',
            num_classes = None,
            model_output = 1280
    ):
        super().__init__()
        self.efficient_net_type = efficient_net_type

        self.efficient_net = EfficientNet.from_pretrained(self.efficient_net_type)
        if num_classes is None:
            self.output_linear = nn.Identity()
        else:
            self.output_linear = nn.Linear(1000, num_classes)

        self.last = nn.Identity()  # ######################################### This will become the PCA layer
        self.activations = list()
        self.mode = 'classification'
        # self.set_hooks()
        self.model_output = model_output

    def forward_activations(self, x):
        self.activations = list() # self.activations[0]: 1x1280
        x = self.efficient_net(x) # 1x1000

        if self.model_output == 1280:
            self.activations[0] = self.last(self.activations[0])
        else:
            x = self.last(x)

        return x, self.activations

    def forward_classification(self, x):
        x = self.efficient_net(x)
        x = self.output_linear(x)

        return x

    def forward(self, x):
        if self.mode == 'classification':
            return self.forward_classification(x)
        elif self.mode == 'activations':
            return self.forward_activations(x)
        else:
            raise NotImplemented('Model recieved unknown mode')

    def get_image_size(self):
        return self.efficient_net.get_image_size(self.efficient_net_type)

    def set_hooks(self):
        def hook(module, inp, out):
            self.activations.append(out.squeeze())

        self.efficient_net._modules['_avg_pooling'].register_forward_hook(hook)

from torchvision import transforms

class Pad:
  """
  A class for padding using torch transforms on numpy array.
  This is highly tailored to this code...
  """
  def __init__(self, value=0, return_size=224):
    self.value = value # Not doing anything. 
    self.return_size = return_size

  def __call__(self,x):
    """
    x:
      numpy array [N x M]
    
    returns:
      numpy array [return_size x return_size]
    """

    ax_0 = (self.return_size - x.shape[0])//2 + 1
    ax_1 = (self.return_size - x.shape[1]) // 2 + 1

    return np.pad(
        x.squeeze(),
        ((ax_0,ax_0), (ax_1, ax_1))
        )[:self.return_size, :self.return_size,np.newaxis]

class Repeat_axis:
  """
  EffNet expects RGB images, we have grayscale
  """
  def __init__(self,axis=-1,num=3):
    self.axis = axis
    self.num = num

  def __call__(self,x):
    # x is [H x W x C] !!!
    return np.repeat(x, self.num, self.axis)

trainvaltest_transforms = transforms.Compose([
  Pad(5),
  Repeat_axis()
])

for snr in snrs:
  print(f"TRAINING SNR TYPE: {snr}, MODEL: fconvnet3")
  folds = 5
  batch_size = 32
  model_folder = "others"
  device = 'cuda'


  ######################
  ######################
  version_number = 1                                    # version number
  model_name = f"effnet_{snr}"                     # name of file
  model = EffNetModel(num_classes=1).to(device)         # model
  ######################
  ######################

  submission =  pd.DataFrame()
  submission['segment_id'] = test_df['segment_id']
  save_name = f'v{version_number}_{model_name}_final'

  for i in range(folds):
      submission[f'fold {i}'] = show_validation_accuracy_two(
          test_df['iq_sweep_burst'].reshape(list(test_df['iq_sweep_burst'].shape) + [1]),
          None, 
          i, 
          model_folder, 
          save_name, 
          model, 
          batch_size, 
          0,
          transforms=trainvaltest_transforms
          )

  submission['prediction'] = submission[[f'fold {j}' for j in range(folds)]].median(1).astype('float')
  for col in submission.drop(['segment_id'], axis='columns').columns:
    output = pred_description(submission[col], test_df['target_type'].astype(int), True)
    print(f"ROC-AUC {col}: {output['ROC_AUC']}")



TRAINING SNR TYPE: low_snr, MODEL: fconvnet3
Loaded pretrained weights for efficientnet-b0
Number of Total Items: 284
ROC-AUC fold 0: 0.8168552709946396
Number of Total Items: 284
ROC-AUC fold 1: 0.8986003573555689
Number of Total Items: 284
ROC-AUC fold 2: 0.8964661504864007
Number of Total Items: 284
ROC-AUC fold 3: 0.8672324796505856
Number of Total Items: 284
ROC-AUC fold 4: 0.8806333134802462
Number of Total Items: 284
ROC-AUC prediction: 0.8876315267024022
TRAINING SNR TYPE: high_snr, MODEL: fconvnet3
Loaded pretrained weights for efficientnet-b0
Number of Total Items: 284
ROC-AUC fold 0: 0.638872344649593
Number of Total Items: 284
ROC-AUC fold 1: 0.7708953742306929
Number of Total Items: 284
ROC-AUC fold 2: 0.7601250744490768
Number of Total Items: 284
ROC-AUC fold 3: 0.7834524518562637
Number of Total Items: 284
ROC-AUC fold 4: 0.7304447091522732
Number of Total Items: 284
ROC-AUC prediction: 0.76508834623784
